In [ ]:
from typing import List

## Leetcode - 3562 - Maximum Profit in Company Hierarchy
class Solution:

    def dfs(self, u: int, tree: List[List[int]], present: List[int], future: List[int], budget: int):
        # Returns (dp_if_parent_skips, dp_if_parent_buys, subtree_max_cost)
        
        # Costs and profits for current node u
        cost_full = present[u]
        profit_full = future[u] - cost_full
        
        cost_half = cost_full // 2
        profit_half = future[u] - cost_half
        
        # Initialize accumulators for children results
        # We start with size 0 (cost 0) and profit 0
        dp_children_if_u_buys = [0] * (budget + 1)
        dp_children_if_u_skips = [0] * (budget + 1)
        
        # Track the max cost possible in the current merged subtree (bounded by budget)
        current_subtree_cost = 0
        
        for v in tree[u]:
            child_res_skips, child_res_buys, child_subtree_cost = self.dfs(v, tree, present, future, budget)
            
            # Optimization: Only iterate up to the valid costs we have seen so far
            # New max cost after merge
            new_subtree_cost = min(budget, current_subtree_cost + child_subtree_cost)
            
            new_dp_buys = [-float('inf')] * (budget + 1)
            new_dp_skips = [-float('inf')] * (budget + 1)
            
            # Initialize base cases (taking 0 from new child)
            # This is effectively copying the old DP state into the new one before merging
            for w in range(current_subtree_cost + 1):
                new_dp_buys[w] = dp_children_if_u_buys[w]
                new_dp_skips[w] = dp_children_if_u_skips[w]

            # Knapsack Merge with bounds
            # w ranges from the new max cost down to 0
            for w in range(new_subtree_cost, -1, -1):
                # Try to allocate k budget to child v
                # k can go up to child_subtree_cost, but w-k must be <= current_subtree_cost
                # So k >= w - current_subtree_cost  =>  k_min = max(0, w - current_subtree_cost)
                # Also k <= child_subtree_cost and k <= w
                
                start_k = max(0, w - current_subtree_cost)
                end_k = min(w, child_subtree_cost)
                
                for k in range(start_k, end_k + 1):
                    # If u buys
                    if dp_children_if_u_buys[w - k] != -float('inf'):
                        val = dp_children_if_u_buys[w - k] + child_res_buys[k]
                        if val > new_dp_buys[w]:
                            new_dp_buys[w] = val
                    
                    # If u skips
                    if dp_children_if_u_skips[w - k] != -float('inf'):
                        val = dp_children_if_u_skips[w - k] + child_res_skips[k]
                        if val > new_dp_skips[w]:
                            new_dp_skips[w] = val
            
            dp_children_if_u_buys = new_dp_buys
            dp_children_if_u_skips = new_dp_skips
            current_subtree_cost = new_subtree_cost

        # Construct final results for node u
        res_parent_skips = [0] * (budget + 1)
        res_parent_buys = [0] * (budget + 1)
        
        # The max cost this subtree can incur includes u's own cost
        # We can bound the final loop by current_subtree_cost + cost_full (or cost_half)
        # But simply iterating to budget is fine as it's O(Budget) per node now
        
        for w in range(budget + 1):
            # Calculate res_parent_skips[w] (Parent of u did NOT buy)
            val_skip = dp_children_if_u_skips[w] if w <= current_subtree_cost else dp_children_if_u_skips[current_subtree_cost]
            
            val_buy = -float('inf')
            if w >= cost_full:
                prev_w = min(w - cost_full, current_subtree_cost)
                if dp_children_if_u_buys[prev_w] != -float('inf'):
                    val_buy = dp_children_if_u_buys[prev_w] + profit_full
            
            res_parent_skips[w] = max(val_skip, val_buy)
            
            # Calculate res_parent_buys[w] (Parent of u DID buy)
            val_buy_half = -float('inf')
            if w >= cost_half:
                prev_w = min(w - cost_half, current_subtree_cost)
                if dp_children_if_u_buys[prev_w] != -float('inf'):
                    val_buy_half = dp_children_if_u_buys[prev_w] + profit_half
                
            res_parent_buys[w] = max(val_skip, val_buy_half)
            
        # Return the actual max cost reachable by this subtree to help parents optimize
        # The max cost is bounded by budget anyway
        total_subtree_cost = min(budget, current_subtree_cost + cost_full) 
        
        return res_parent_skips, res_parent_buys, total_subtree_cost
    
    def maxProfit(self, n: int, present: List[int], future: List[int], hierarchy: List[List[int]], budget: int) -> int:
        
        tree = [[] for _ in range(n)]
        has_parent = [False] * n

        for u, v in hierarchy:
            tree[u - 1].append(v - 1)
            has_parent[v - 1] = True

        root = -1
        for i in range(n):
            if not has_parent[i]:
                root = i
                break
        
        if root == -1:
            return 0
            
        # Run DFS from root
        res_skips, _, _ = self.dfs(root, tree, present, future, budget)
        
        return max(0, max(res_skips))

In [ ]:
from typing import List
from collections import defaultdict

class Solution:
    def containsDuplicate(self, nums: List[int]) -> bool:
        n = len(nums)
        dict = defaultdict(int)
        if (n==0):
            return False
        for i in range(n):
            dict[nums[i]] += 1
        res = 0
        for key in dict:
            if dict[key]>1:
                res +=1
        return True if res > 0 else False

if __name__ == "__main__":
    sol = Solution()
    print(sol.containsDuplicate([1,2,3,1]))  # True
    print(sol.containsDuplicate([1,2,3,4]))  # False
    print(sol.containsDuplicate([1,1,1,3,3,4,3,2,4,2]))  # True

CompilationException: [snippet: [from typing import List
from collections import defaultdict

class Solution:
    def containsDuplicate(self, nums: List[int]) -> bool:
        n = len(nums)
        dict = defaultdict(int)
        if (n==0):
            return False
        for i in range(n):
            dict[nums[i]] += 1
        res = 0
        for key in dict:
            if dict[key]>1:
                res +=1
        return True if res > 0 else False

if __name__ == "__main__":
    sol = Solution()
    print(sol.containsDuplicate([1,2,3,1]))  # True
    print(sol.containsDuplicate([1,2,3,4]))  # False
    print(sol.containsDuplicate([1,1,1,3,3,4,3,2,4,2]))  # True], status: REJECTED]

In [ ]:
class Solution {
    public boolean containsNearbyAlmostDuplicate(int[] nums, int id, int vd) {
        int n = nums.length;
        TreeSet<Long> set = new TreeSet<>();
        Map<Integer, Integer> cnt = new HashMap<>();
        int l = 0;
        for (int i = 0; i < n; i++) {
            long x = (long) (nums[i] - vd);
            Long ceil = set.ceiling(x);
            if (ceil!=null && ceil<=x+vd) {
                return true;
            }
            set.add((long)nums[i]);
            cnt.computeIfAbsent(nums[i], k->cnt.getOrDefault(k, 0)+1);
            if (i>=id) {
                if (cnt.get(nums[i-id])==1) {
                    set.remove((long)(nums[i-id]));
                    cnt.remove(nums[i]);
                } else {
                    cnt.put(nums[i-id], cnt.getOrDefault(nums[i], 0)-1);
                }
            }
        }
        return false;
    }
}




In [2]:
from typing import List

class Solution:
    def missingNumber(self, nums: List[int]) -> int:
        n = len(nums)
        if nums is not None:
            return 0
        sum = 0
        for i in range(n):
            sum+=nums[i]
        print(sum)
        sum = ((n * (n+1)) // 2) - sum
        return sum

if __name__ == "__main__":
    sol = Solution()
    print(sol.missingNumber([3,0,1]))  # 2
    print(sol.missingNumber([0,1]))    # 2
    print(sol.missingNumber([9,6,4,2,3,5,7,0,1]))  # 8

0
0
0


In [ ]:
class Solution:
    
    def minSwapsCouples(self, row: List[int]) -> int:
        n = len(row)
        ans = 0
        seat = {}

        for i in range(n):
            seat[row[i]] = i

        for i in range(0, n, 2):
            first = row[i]
            second = first ^ 1
            
            if row[i+1] != second:
                ans += 1
                
                seat_of_second = seat[second]
                
                person_at_wrong_seat = row[i+1]
                
                row[i+1], row[seat_of_second] = row[seat_of_second], row[i+1]
                
                seat[person_at_wrong_seat] = seat_of_second
        
        return ans

if __name__ == "__main__":
    sol = Solution()
    print(sol.minSwapsCouples([0, 2, 1, 3])) # 1
    print(sol.minSwapsCouples([3, 2, 0, 1])) # 0

In [ ]:
from typing import List


class Solution:
    def binarySearch(self, nums: List[int], target: int) -> int:
        left, right = 0, len(nums) - 1
        while left <= right:
            mid = left + (right - left) // 2
            if nums[mid] == target:
                return mid
            elif nums[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return -1

    def maxProfit(self, k: int, prices: List[int]) -> int:
        n = len(prices)
        if n == 0:
            return 0

        # If k is larger than n/2, we can make as many transactions as we want
        if k >= n // 2:
            max_profit = 0
            for i in range(1, n):
                if prices[i] > prices[i - 1]:
                    max_profit += prices[i] - prices[i - 1]
            return max_profit

        # DP table where dp[i][j] represents the max profit using at most i transactions up to day j
        dp = [[0] * n for _ in range(k + 1)]

        for i in range(1, k + 1):
            max_diff = -prices[0]
            for j in range(1, n):
                dp[i][j] = max(dp[i][j - 1], prices[j] + max_diff)
                max_diff = max(max_diff, dp[i - 1][j] - prices[j])

        return dp[k][n - 1]


if __name__ == "__main__":
    s = Solution()
    print(s.maxProfit(2, [3, 2, 6, 5, 0, 3]))  # Expected output: 7

In [ ]:
class Solution:
    def lower_bound(self, nums: List[int], target: int) -> int:
        if not nums:
            return -1
        left, right = 0, len(nums) - 1
        while left <= right:
            mid = left + (right - left) // 2
            if nums[mid] < target:
                left = mid + 1
            else:
                right = mid - 1
        return left
    def upper_bound(self, nums: List[int], target: int) -> int:
        if not nums:
            return -1
        left, right = 0, len(nums) - 1
        while left <= right:
            mid = left + (right - left) // 2
            if nums[mid] <= target:
                left = mid + 1
            else:
                right = mid - 1
        return right
    def countPalindromicSubsequence(self, s: str) -> int:
        map = {}
        n = len(s)
        for i in range(n):
            if s[i] not in map:
                map.update({s[i]: [i]})
            else:
                map[s[i]].append(i)

        visited = set()
        for i in range(1, n - 1):
            for j in range(26):
                ch = chr(ord('a') + j)
                left = self.lower_bound(map.get(ch, []), i)
                right = self.upper_bound(map.get(ch, []), i)
                if 

                


In [ ]:
class Solution {

    class Cells {
        int x;
        int y;
        int obstacles;

        public Cells(int x, int y, int obstacles) {
            this.x = x;
            this.y = y;
            this.obstacles = obstacles;
        }
    }

    public int minimumObstacles(int[][] grid) {
        int n = grid.length;
        int m = grid[0].length;
        int[][] dp = new int[n][m];
        PriorityQueue<Cells> pq = new PriorityQueue<>((a, b) -> a.obstacles - b.obstacles);
        for (int i = 0; i < n; i++) {
            Arrays.fill(dp[i], Integer.MAX_VALUE);
        }
        dp[0][0] = 0;
        pq.offer(new Cells(0, 0, 0));
        int[][] directions = {{1, 0}, {-1, 0}, {0, 1}, {0, -1}};
        int mini = Integer.MAX_VALUE;
        while (!pq.isEmpty()) {
            Cells cell = pq.poll();
            int x = cell.x;
            int y = cell.y;
            int obstacles = cell.obstacles;
            if (x == n - 1 && y == m - 1) { 
                mini = Math.min(mini, obstacles);
            }
            for (int[] dir : directions) {
                int newX = x + dir[0];
                int newY = y + dir[1];
                if (newX >= 0 && newX < n && newY >= 0 && newY < m) {
                    int newObstacles = obstacles + grid[newX][newY];
                    if (newObstacles < dp[newX][newY]) {
                        dp[newX][newY] = newObstacles;
                        pq.offer(new Cells(newX, newY, newObstacles));
                    }
                }
            }
        }
        return mini;
    }
}



In [ ]:
from typing import List


class Solution:
    def isSafeBoundry(self, x: int, y: int, n: int, m: int) -> bool:
        return 0 <= x < n and 0 <= y < m

    def shortestPath(self, grid: List[List[int]], k: int) -> int:
        n = len(grid)
        m = len(grid[0])
        dp = [[[float("inf")] * (k + 1) for _ in range(m)] for _ in range(n)]
        dp[0][0][0] = 0
        queue = [(0, 0, 0)]  # (x, y, obstacles eliminated)
        directions = [(1, 0), (-1, 0), (0, 1), (0, -1)]
        while queue:
            x, y, elim = queue.pop(0)
            for dx, dy in directions:
                newX, newY = x + dx, y + dy
                if self.isSafeBoundry(newX, newY, n, m):
                    newElim = elim + grid[newX][newY]
                    if newElim <= k and dp[newX][newY][newElim] > dp[x][y][elim] + 1:
                        dp[newX][newY][newElim] = dp[x][y][elim] + 1
                        queue.append((newX, newY, newElim))
        ans = min(dp[n - 1][m - 1])
        return ans if ans != float("inf") else -1


if __name__ == "__main__":
    s = Solution()
    print(
        s.shortestPath([[0, 0, 0], [1, 1, 0], [0, 0, 0], [0, 1, 1], [0, 0, 0]], 1)
    )  # Expected output: 6


In [ ]:
class Solution {
    public int countPartitions(int[] nums, int k) {
        int n = nums.length;
        int mod = ((int) 1e9)+ 7;
        int ans = 0;
        int[] dp = new int[n+1];
        Arrays.fill(dp, Integer.MAX_VALUE);
        int[] prefixSum = new int[n+1];
        TreeMap<Integer, Integer> treeMap = new TreeMap<>();
        Arrays.fill(prefixSum, 0);
        dp[0] = 1;
        prefixSum[0] = 0;
        for(int i=0, j=0;i<n;i++) {
            treeMap.put(nums[i], treeMap.getOrDefault(nums[i], 0) + 1);
            while(j<=i && treeMap.lastKey() - treeMap.firstKey() > k) {
                treeMap.put(nums[j], treeMap.get(nums[j]) - 1);
                if (treeMap.get(nums[j]) == 0) {
                    treeMap.remove(nums[j]);
                }
                j++;
            }
            dp[i+1] = (prefixSum[i] - prefixSum[j-1] + mod) % mod;
            prefixSum[i+1] = (prefixSum[i] + dp[i+1]) % mod;
        }
        return dp[n];
    }
}



In [ ]:
from sortedcontainers import SortedDict

class Solution:
    def countPartitions(self, nums: List[int], k: int) -> int:
        n = len(nums)
        mod = int(1e9) + 7
        ans = 0
        dp = [0] * (n + 1)
        dp[0] = 1
        prefix_sum = [0] * (n + 1)
        prefix_sum[0] = 1
        sorted_dict: SortedDict = SortedDict()
        j = 0
        for i in range (n):
            sorted_dict[nums[i]] = sorted_dict.get(nums[i], 0) + 1
            while j<=i and sorted_dict.peekitem(-1)[0] - sorted_dict.peekitem(0)[0] > k:
                sorted_dict[nums[j]] -= 1
                if sorted_dict[nums[j]] == 0:
                    del sorted_dict[nums[j]]
                j += 1
            dp[i+1] = (prefix_sum[i] - prefix_sum[j-1] + mod) % mod
            prefix_sum[i+1] = (prefix_sum[i] + dp[i+1]) % mod
        
        return dp[n]

In [ ]:
# Leetcode - 1975 - Maximum Matrix Sum
class Solution:
    def maxMatrixSum(self, matrix: List[List[int]]) -> int:
        cnt = 0
        n = len(matrix)
        m = len(matrix[0])

        min_abs = float('inf')
        total_sum = 0
        for i in range(n):
            for j in range(m):
                val = matrix[i][j]
                total_sum += abs(val)
                if val < 0:
                    cnt += 1
                min_abs = min(min_abs, abs(val))
        
        if cnt % 2 == 0:
            return total_sum
        else:
            return total_sum - 2 * min_abs

In [ ]:
# Leetcode - 1161 - Maximum Level Sum of a Binary Tree
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def maxLevelSum(self, root: Optional[TreeNode]) -> int:
        if not root:
            return 0
        from collections import deque
        queue = deque([root])
        max_sum = float('-inf')
        level = 1
        result_level = 1
        while queue:
            level += 1
            level_size = len(queue)
            current_level_sum = 0
            while level_size > 0:
                cuurent_node = queue.popleft()
                current_level_sum += cuurent_node.val
                if cuurent_node.left:
                    queue.append(cuurent_node.left)
                if cuurent_node.right:
                    queue.append(cuurent_node.right)
                level_size -= 1
            if current_level_sum > max_sum:
                max_sum = current_level_sum
                result_level = level - 1
        return result_level


In [ ]:
# Leetcode - 1975 - Kth Largest Level Sum in a Binary Tree
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def kthLargestLevelSum(self, root: Optional[TreeNode], k: int) -> int:
        if not root:
            return 0
        
        level_sum_list = []
        from collections import deque
        queue = deque([root])
        
        while queue:
            curr_level_sum = 0
            curr_level_size = len(queue)
            while curr_level_size > 0:
                current_node = deque.popleft()
                curr_level_size -= 1
                curr_level_sum += current_node.val
                if current_node.left:
                    deque.append(current_node.left)
                if current_node.right:
                    deque.append(current_node.right)
            level_sum_list.append(curr_level_sum)

        level_sum_list.sort(reverse=True)
        if k <= len(level_sum_list):
            return level_sum_list[k - 1]
        else:
            return -1

In [ ]:
# Leetcode - 1339 - Maximum Product of Splitted Binary Tree
# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    max_product = 0
    total_sum = 0
    def dfs(self, root) -> int:
        if not root:
            return 0
        left_sum = self.dfs(root.left)
        right_sum = self.dfs(root.right)
        current_sum = left_sum + right_sum + root.val
        complement_sum = self.total_sum - current_sum
        product = current_sum * complement_sum
        self.max_product = max(self.max_product, product)
        return current_sum
        
    def maxProduct(self, root: Optional[TreeNode]) -> int:
        if not root:
            return 0
        mod = 10**9 + 7
        total_sum = 0
        queue = deque([root])
        while not queue:
            current_node = queue.pop_left()
            total_sum += current_node.val
            if current_node.left:
                queue.append(current_node.left)
            if current_node.right:
                queue.append(current_node.right)
        
        print(total_sum)
        self.total_sum = total_sum
        self.dfs(root)
        return self.max_product % mod
